In [1]:
!pip install -U git+https://github.com/huggingface/transformers -q

In [3]:
!wget https://d8it4huxumps7.cloudfront.net/files/66e31d6ee96cd_student_resource_3.zip
!unzip 66e31d6ee96cd_student_resource_3.zip
!rm 66e31d6ee96cd_student_resource_3.zip

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-09-13 17:39:59--  https://d8it4huxumps7.cloudfront.net/files/66e31d6ee96cd_student_resource_3.zip
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 108.139.15.116, 108.139.15.41, 108.139.15.75, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|108.139.15.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5184192 (4.9M) [application/zip]
Saving to: '66e31d6ee96cd_student_resource_3.zip'

66e31d6ee96cd_stude 100%[===================>]   4.94M  --.-KB/s    in 0.1s    

2024-09-13 17:39:59 (49.0 MB/s) - '66e31d6ee96cd_student_resource_3.zip' saved [5184192/5184192]

Archive:  66e31d6ee96cd_student_resource_3.zip
   creating: student_resource 3/
  inflating: __MACOSX/._student_resource 3  
  inflating: student_resource 3/sample_code.py  
  inflating: __MACOSX/student_resource 3/._sample_code.py  
  inflating: student_resource 3/.DS_Store  
  inflating: __MACOSX/student_resource 3/._.DS_Store  
   creating: 

In [113]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import pandas as pd
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import os
from tqdm import tqdm


In [5]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

english = {}


for k in entity_unit_map.keys():
    english[k] = k
    
english["item_weight"] = "weight"
english["item_volume"] = "volume"


# Setup

In [38]:
df = pd.read_csv("/kaggle/working/student_resource 3/dataset/train.csv")

In [6]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

# Inference

In [140]:
trans = transforms.Resize((600, 600))
def get_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    resized = trans(img)
    return resized

In [141]:
def url_to_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    resized = trans(img)
    return resized

# Replace the 'image_link' column with PIL.Image objects


In [142]:
def get_message(entity_name):
    message = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {"type": "text", "text": f"What is the {english[entity_name]} of the item in the image"},
                ],
            }
        ]
    return message

In [161]:
from typing import List
def forward(entity_names, images):
    """
    entity_name -> It is a list of entity names in the batch
    images -> List of images
    """
    
    messages = [ get_message(entity)  for entity in entity_names]
    
    texts = [
        processor.apply_chat_template(msg, add_generation_prompt=True) 
        for msg in messages ]

    
    #text = processor.apply_chat_template(messages, add_generation_prompt=True)

    
    inputs = processor(
        text=texts, 
        images=images, 
        padding=True, 
        return_tensors="pt"
    )

    inputs = inputs.to("cuda")
    
    
    output_ids = model.generate(**inputs, max_new_tokens=32)
    
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    del inputs
    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    del generated_ids
    del output_ids

    
    
    
    return output_text

In [162]:
import re

# Dictionary to map unit abbreviations to full forms
unit_mapping = {
    'g': 'gram',
    'kg': 'kilogram',
    'mg': 'milligram',
    'ug': 'microgram',
    'lb': 'pound',
    'oz': 'ounce',
    'ton': 'ton',
    'mm': 'millimeter',
    'cm': 'centimeter',
    'm': 'meter',
    'ft': 'foot',
    'in': 'inch',
    'yd': 'yard'
}

# Function to convert the sentence
def extract_measurement(sentence):
    # Regular expression to match number and unit
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', sentence)
    
    if match:
        number = match.group(1)
        unit = match.group(3)
        
        # Convert unit to full form
        full_unit = unit_mapping.get(unit, unit)
        
        # Form the final string
        result = f"{number} {full_unit}"
        return result
    else:
        return None

# Example usage
sentence_1 = "The weight of the ball is 200 g."
sentence_2 = "The width is 5mm."

print(extract_measurement(sentence_1))  # Output: 200 gram
print(extract_measurement(sentence_2))  # Output: 5 millimeter


200 gram
5 millimeter


In [163]:
subset = df[20:30]

In [164]:
imgs = []
enames = []
values = []

In [165]:
for ix in range(21, 26):
    img = get_image(subset.loc[ix, "image_link"])
    ename = subset.loc[ix, "entity_name"]
    value = subset.iloc[ix - 20]["entity_value"]
    
    imgs.append(img)
    enames.append(ename)
    values.append(value)

In [166]:
import time 

start = time.time()
output = forward(enames, imgs)
end = time.time()

In [167]:
execution_time = end - start  # Calculate the difference
print(f"Execution time: {execution_time} seconds")


Execution time: 21.175172328948975 seconds


In [168]:
for x, y in zip(output, values):
    print("Predicted: " + extract_measurement(x) + " ------" + "Actual: " + y)

Predicted: 22.2 inches ------Actual: 4.0 gallon
Predicted: 2.7 grams ------Actual: 2.7 gram
Predicted: 112 grams ------Actual: 112 gram
Predicted: 4.1 kilogram ------Actual: 4.1 kilogram
Predicted: 48 V ------Actual: 48.0 volt


In [105]:
output = forward(ename, img)[0]
print(output)
print("Predicted: " + extract_measurement(output))
print("Actual: " + value)

The weight of the item in the image is 112 grams.
Predicted: 112 grams
Actual: 112 gram


In [108]:
cmp = pd.DataFrame({"index": [], "entity_name"=[] ,"prediction": [], "actual": []})

In [125]:
for index, record in tqdm(subset.iterrows(), total=len(subset)):
    img = url_to_image(record["image_link"])
    ename = record["entity_name"]
    value = record["entity_value"]
    
    pred = extract_measurement(forward(ename, img)[0])
    
    cmp.loc[len(cmp)] = {"entity_name": ename, "prediction": pred, "actual": value}

100%|██████████| 10/10 [00:34<00:00,  3.42s/it]


In [123]:
cmp

,index,prediction,actual
-1,NaN,18.55 grams,18.55 gram
1,NaN,18.55 grams,18.55 gram
2,NaN,18.55 grams,18.55 gram
3,NaN,9.4 inches,200 gram
4,NaN,22.2 inches,4.0 gallon
5,NaN,2.7 grams,2.7 gram
6,NaN,112 grams,112 gram
7,NaN,4.1 kilogram,4.1 kilogram
8,NaN,48 V,48.0 volt
9,NaN,158 gram,158.0 gram


In [120]:
cmp.loc[len(cmp)] = {"prediction": pred, "actual": value}

In [121]:
cmp

,index,prediction,actual
-1,NaN,18.55 grams,18.55 gram
1,NaN,18.55 grams,18.55 gram
2,NaN,18.55 grams,18.55 gram
